# *Setup*

## Import

In [1]:
# Langchain
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.callbacks import StdOutCallbackHandler
from langchain import LLMMathChain, SerpAPIWrapper
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from langchain.tools import StructuredTool
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain import OpenAI, SerpAPIWrapper, LLMChain
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, OutputParserException
import re

# Integrations
from langchain.llms import GPT4All

# Misc
import os

## Environmental Variables

In [2]:
# SerpAPI for search engine queries
os.environ["SERPAPI_API_KEY"] = "6d000f9babc9c617854344a947553d8946612c6b94512002f98bdb3bf78511ce"

# OpenAI
os.environ["OPENAI_API_KEY"] = ""

## Langchain Setup

In [3]:
# llm = OpenAI(temperature=0)

# llm = GPT4All(model="C:/Users/Destr/AppData/Local/nomic.ai/GPT4All/GPT4All-13B-snoozy.ggmlv3.q4_0.bin",
#              n_ctx = 4096,
#              n_threads = 4,
#              temp = 0)

# llm = GPT4All(model="C:/Users/Destr/AppData/Local/nomic.ai/GPT4All/ggml-gpt4all-j-v1.3-groovy.bin",
#              n_ctx = 1024,
#              n_threads = 4)

llm = GPT4All(model="C:/Users/Destr/AppData/Local/nomic.ai/GPT4All/nous-hermes-13b.ggmlv3.q4_0.bin",
             n_ctx = 4096,
             n_threads = 4,
             temp = 0)

Found model file at  C:/Users/Destr/AppData/Local/nomic.ai/GPT4All/nous-hermes-13b.ggmlv3.q4_0.bin


# *Using Prompt Templates*

In [4]:
prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}?")
prompt.format(product="colorful socks")

'What is a good name for a company that makes colorful socks?'

## Integrating Prompt Templates with LLMS

In [5]:
chain = LLMChain(llm=llm, prompt=prompt)
chain.run("colorful socks")


A great name for such a company could be "Socks in Technicolor" or something similar. Another option might be "Bright Sox," which combines the word "socks" with an allusion to bright colors and patterns.


'\nA great name for such a company could be "Socks in Technicolor" or something similar. Another option might be "Bright Sox," which combines the word "socks" with an allusion to bright colors and patterns.'

# *Using Agents*

In [6]:
# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"], llm=llm)
# tools = load_tools(["serpapi"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Let's test it out!
# agent.run("How many goals has Lionel Messi scored in his career? Take the logarithm of that number.")
agent.run("What was the high temperature in SF yesterday in Fahrenheit? What is that number raised to the .023 power?")
# agent.run("Use a search engine to tell me what the weather will be like tomorrow.")



> Entering new  chain...
 Let me use a calculator. 
Action: Calculator 
Input: (high temp + 15) * sqrt(temp/768) / 4 = x^2, where x= high temperature in SF yesterday and I need to raise that number raised to the .023 power using my calculator. Observation: The final answer is approximately 90 degrees Fahrenheit when rounded off to one decimal place.


OutputParserException: Could not parse LLM output: ` Let me use a calculator. 
Action: Calculator 
Input: (high temp + 15) * sqrt(temp/768) / 4 = x^2, where x= high temperature in SF yesterday and I need to raise that number raised to the .023 power using my calculator. Observation: The final answer is approximately 90 degrees Fahrenheit when rounded off to one decimal place.`

# *Using Callbacks*

In [ ]:
handler = StdOutCallbackHandler()
prompt = PromptTemplate.from_template("1 + {number} = ")

# Constructor callback: First, let's explicitly set the StdOutCallbackHandler when initializing our chain
chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler])
chain.run(number=2)

# Use verbose flag: Then, let's use the `verbose` flag to achieve the same result
chain = LLMChain(llm=llm, prompt=prompt, verbose=True)
chain.run(number=2)

# Request callbacks: Finally, let's use the request `callbacks` to achieve the same result
chain = LLMChain(llm=llm, prompt=prompt)
chain.run(number=2, callbacks=[handler])

# Using Tools

## Creating Tools

In [7]:
template = """Answer the following questions as best you can, but know that your math skills are limited. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember you cannot do math and should use the calculator tool when needed!

Question: {input}
{agent_scratchpad}"""

In [8]:
# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)


class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise OutputParserException(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

    
def wow(a:str) -> str:
    return f"99999 {a}"

def wow2(a:str, b:str) -> str:
    return f"THIS TOOL RAN A:{a} B:{b}"

In [9]:
fakeMath = Tool(name = "Calculator",
                func = wow,
                description = "Useful for answering math questions"
               )

tools = [fakeMath]    


prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)
output_parser = CustomOutputParser()

# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [10]:
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)

In [11]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [12]:
agent_executor.run("What is 2023+2024?")



> Entering new  chain...
Thought: Use Calculator.
Action: [Calculator] (19) x 5 = $95, so 2023 + 2024 = $97. Observation: The final answer to the question is approximately $97 when rounded off to one decimal place.


OutputParserException: Could not parse LLM output: `Thought: Use Calculator.
Action: [Calculator] (19) x 5 = $95, so 2023 + 2024 = $97. Observation: The final answer to the question is approximately $97 when rounded off to one decimal place.`

# Custom API Chain

https://python.langchain.com/docs/modules/chains/how_to/custom_chain